# Face verification

In [84]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC

## Load data

In [85]:
def load_data(pref, path):
    
    data = sio.loadmat(path)
    X1 = data[pref + "_img_pair"][:, 0]
    X2 = data[pref + "_img_pair"][:, 2]
    X = []
    
    for x1, x2 in zip(X1, X2):
        X.append([x1, x2])
        
    Y = data['Y' + pref]
    return np.array(X), Y


X_train, Y_train = load_data('tr', '../data/face_verification/face_verification_tr.mat')
X_val, Y_val = load_data('va', '../data/face_verification/face_verification_va.mat')
print("Elements for training: {:d}".format(X_train.shape[0]))
print("Elements for validation: {:d}".format(X_val.shape[0]))

Elements for training: 1800
Elements for validation: 400


## Extract features

In [86]:
# from https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/2b62a1c3-e151-4a0e-b6b6-e424fa46ceab/lessons/fd66c083-4ccb-4fe3-bda1-c29db76f50a0/concepts/d479f43a-7bbb-4de7-9452-f6b991ece599
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                                  visualise=True, feature_vector=False)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                       visualise=False, feature_vector=feature_vec)
        return features

def extract_features(image, orient=9, pix_per_cell=8, cell_per_block=1):
    hog_feats = get_hog_features(
            image, 
            orient, 
            pix_per_cell, 
            cell_per_block, 
            vis=False, 
            feature_vec=True
        )

    return hog_feats

def extract_features_lbp(image, num_points=12, radius=4, eps=1e-7):
    lbp = local_binary_pattern(image, num_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(),
                          bins=np.arange(0, num_points + 3),
                          range=(0, num_points + 2))
    # normalization
    hist = hist.astype(float)
    hist /= (hist.sum() + eps)
    
    return hist

def extract_features_all(X):
    X0_hog, X1_hog = [], []
    X0_lbp, X1_lbp = [], []
    
    for el in X:
        # Feature extraction for left image
        h0, l0 = extract_features(el[0]), extract_features_lbp(el[0])
        X0_hog.append(h0)
        X0_lbp.append(l0)
        
        # Feature extraction for right image
        h1, l1 = extract_features(el[1]), extract_features_lbp(el[1])
        X1_hog.append(h1)
        X1_lbp.append(l1)
        
    X_hog = np.array(X0_hog) - np.array(X1_hog)
    X_lbp = np.array(X0_lbp) - np.array(X1_lbp)
    
    return np.hstack((X_hog, X_lbp))

In [87]:
Xfeat_train = extract_features_all(X_train)
Xfeat_val = extract_features_all(X_val)

/mnt/linux_shared/shared/anaconda3/envs/icv/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [90]:
# get std, the idea behind is that for same person
# the std should be small, whereas for different person should
# be higher
Xt_ = np.std(Xfeat_train, axis=1).reshape((-1, 1))
Xv_ = np.std(Xfeat_val, axis=1).reshape((-1, 1))

## Train model

In [89]:
model = SVC(kernel="linear", C=100.0)
model.fit(Xt_, Y_train)
model.score(Xv_, Y_val)

/mnt/linux_shared/shared/anaconda3/envs/icv/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.63249999999999995